In [ ]:
from utils.OBJ_helper import OBJ
import os
import numpy as np
import trimesh
from utils.pickel_io import load_from_memory
from utils.Blendshape import ZeroMeanDefMatrix
from scipy import sparse
from scipy import linalg
from scipy.sparse.linalg import splu
from utils.Geodesic_dist import compute_topological_laplacian
from utils.vis_tools import VisPointsAttributes
save_path = "../dataset/multiface/tracked_mesh"

# Heat method to compute distance function $\phi$
- Compute Laplacian
- Algorithm (heat method)
1. Integrate the heat flow
$$(A-tLc)u_t = u_0$$
2. Evaluate the vector field
$$\Chi = - \frac{\nabla u}{|\nabla u|} $$
3. Solve the Poisson equation
$$\Delta \phi = \nabla \Chi$$

## load obj file using original loader

In [ ]:
# # Get neutral face vertices and neutral face triangle list
# target_obj = OBJ(filename = os.path.join(save_path, "sample.obj"))

# list_vertices = target_obj.vertices
# list_triangles = target_obj.faces

# verts = np.asarray(list_vertices)

# CENTER = 3567

# tris = []
# for triangle in list_triangles:
#     # 0: triangle index list
#     # 1: normals
#     # 2: texture coordinate
#     # 3: material configuration
#     tris.append(triangle[0])
# tris = np.asarray(tris)

## load obj file using trimesh loader

In [ ]:
mesh = trimesh.load(os.path.join(save_path, "sample.obj"), force='mesh')
list_vertices = mesh.vertices
list_triangles = mesh.faces

verts = np.asarray(list_vertices)
tris = np.asarray(list_triangles)

CENTER = 2658

In [ ]:
# verts: (N, 3) array (float)
# tris: (m, 3) array (int): indices into the verts array
print(tris.shape)
print(verts.shape)

# Triangle list conversion
- index should be start from 0 to #num_vertices

In [ ]:
if tris.min() > 0:
    for triangle in tris:
        for i in range(3):
            # print(triangle)
            triangle[i] = triangle[i] - int(1)
            

Test vector normalization and func for getting norm of vector

In [ ]:
from utils.common_utils import vecnorm, normalized
dummy = np.array([2, 1, 3])
dummy_norm = vecnorm(dummy)
normalized_dummy = normalized(dummy)
print(dummy_norm)
print(normalized_dummy)

# Compute mesh laplacian
- To compute descritize laplasian, we need to somehow compute weights to obtain a laplacian at a vertex
- Since there are desired properties for discrete laplacian, there are several forms of computation of mesh laplacian
- The one which fulfills all desired properties is __convex weights__
$$w_{ij} = \frac{tan(\frac{\theta_{ij}^1}{2}) + tan(\frac{\theta_{ij}^2}{2})}{2}$$
- The one which fulfills almost all desired properties is __cotangent weight__
$$w_{ij} = \frac{1}{2}(cot(\alpha_{ij}) + cot(\beta_{ij}))$$

In [ ]:
N = len(verts)
print(f"Number of vertecies: {N}")

In [ ]:
W_ij = np.empty(0, np.double)
I = np.empty(0, np.int32)
J = np.empty(0, np.int32)

In [ ]:
for i1, i2, i3 in [(0, 1, 2), (1, 2, 0), (2, 0, 1)]:
    vi1 = tris[:, i1]
    vi2 = tris[:, i2]
    vi3 = tris[:, i3]
    u = verts[vi2] - verts[vi1]
    v = verts[vi3] - verts[vi1]
    cotan = (u * v).sum(axis = 1) / vecnorm(np.cross(u, v))
    W_ij = np.append(W_ij, 0.5 * cotan)
    I = np.append(I, vi2)
    J = np.append(J, vi3)
    W_ij = np.append(W_ij, 0.5 * cotan)
    I = np.append(I, vi3)
    J = np.append(J, vi2)

In [ ]:
print(f"num of indecies: {len(vi1)}")
print(f"num of indecies: {len(vi2)}")
print(f"num of indecies: {len(vi3)}")

In [ ]:
print(max(vi1))
print(max(vi2))
print(max(vi3))

In [ ]:
type(verts)
print(verts.shape)
print(tris.shape)

# make sparse matrix for convex weight
- N = num_verts
- T = num_faces 
- W_ij = [Tx6,] (T x 6)
    - Each element has cotanjent at the angle which opposite to the i---j
  
                 -
                b_ij
              -     -
            i- - - - -j
              -     -
                a_ij
                 -
  
- I = [Tx6] index list for row
- J = [Tx6] index list for column
- L = shape[N, N]

In [ ]:
# Create a sparse matrix
L = sparse.csr_matrix((W_ij, (I, J)), shape = (N, N))


In [ ]:
# TODO: why it is necessary
L = L - sparse.spdiags(L * np.ones(N), 0, N, N)


sum_zero = 0
L = L.toarray()
print(L.shape)

for i in range(L.shape[1]):
    if L[i][i] == 0:
    #     print(i)
        L[i][i] = -1

print(sum_zero)

# convert matrix L into Compressed Sparse Row format
L = sparse.csr_matrix(L)

# Creat a area matrix
- Triangle area can be computed by cross product of two vectors
$$u \times v = ||u||||v||sin(\theta)$$
$$\text{area}_i = \frac{1}{2}||u \times v||$$

In [ ]:
# triangle area matrix
# edges
e1 = verts[tris[:, 1]] - verts[tris[:, 0]]
e2 = verts[tris[:, 2]] - verts[tris[:, 0]]
# compute normal at each triangle
n = np.cross(e1, e2)
# triangle area can be computed by 
triangle_area = .5 * vecnorm(n)

In [ ]:
# compute per-vertex area
vertex_area = np.ones(len(verts))
# triangle area
ta3 = triangle_area / 3
# construct vertex area matrix VA
for i in range(tris.shape[1]): # 0, 1, 2
    # count the number of occurence of index
    bc = np.bincount(tris[:, i].astype(int), ta3)
    print(bc.shape)
    vertex_area += bc
    print(vertex_area)

# Diagonal matrix
# diagonal entries are (#triangles)*1/3 of triangle area
# #triangles = the number of triangles which consists of the vertex i

print(vertex_area.shape)
VA = sparse.spdiags(vertex_area, 0, len(verts), len(verts))
print(VA.toarray())

Check if the matrix A is invertible

In [ ]:
# print(linalg.det(VA.toarray()))

# compute Geodesic distance

Precomputation

In [ ]:
# compute edges
e01 = verts[tris[:, 1]] - verts[tris[:, 0]]
e12 = verts[tris[:, 2]] - verts[tris[:, 1]]
e20 = verts[tris[:, 0]] - verts[tris[:, 2]]

# normalized edges
Ne01 = normalized(e01)
Ne12 = normalized(e12)
Ne20 = normalized(e20)

# triangle area
_triangle_area = 0.5 * vecnorm(np.cross(e01, e12))

unit_normal = normalized(np.cross(Ne01,Ne12))

_unit_normal_cross_e01 = np.cross(unit_normal, e01)
_unit_normal_cross_e12 = np.cross(unit_normal, e12)
_unit_normal_cross_e20 = np.cross(unit_normal, e20)

In [ ]:
# parameters for heat method
h = np.mean([vecnorm(e01), vecnorm(e12), vecnorm(e20)])
m = 1e2
t = m * h ** 2

# Heat method
- step 1: compute heat at time t $u_t$
    - Heat flow can be obtained by solving the symmetric positive-definite system

$$(A-tLc)u_t = u_0$$
$$\text{where } u_0 = \begin{cases} 1 & \text{if $v_i \in \gamma$ (heat source)} \\
0 & \text{otherwise} \end{cases}$$

In [ ]:
u_0 = np.zeros_like(verts)
print(u_0.shape)

Assign heat to a vertex to the top of nose (#3567)

In [ ]:
u_0[CENTER] = 1 # Nose

Solve linear equation via LU decomposition
$$Ax = b$$
$$PA = LU \text{ where $P$ is permutation matrix}$$
$$Ly = Pb$$
$$Ux = y$$


# Solve linear system
- Since some diagonal entries in L and A are zero, matrix L and A are singular
- To solve this linear system, we need to compute pseudo inverse
$$B = (A-tLc)$$
$$Bu_t = u_0$$

- Solve by LU decomposition
$$Ly = u_0$$
$$Uu_t = y$$

In [ ]:
Lc = L
A = VA
B = (A-t*Lc)

# LU decomposition
- B should be invertible
- Such a decomposition is often useful for solving many simultaneous equations where the left-hand side does not change but the right-hand side does

In [ ]:
factored_AtLc = splu(B.tocsc())
u_t = factored_AtLc.solve(u_0)

In [ ]:
VisPointsAttributes(points=verts, attributes=u_t)

# Heat method step2: Compute gradient field $\Chi$

In [ ]:
grad_u = (1 / (2 * _triangle_area)[:, None]) * (
    _unit_normal_cross_e01 * u_t[tris[:, 2]]
    + _unit_normal_cross_e12 * u_t[tris[:, 0]]
    + _unit_normal_cross_e20 * u_t[tris[:, 1]]
)

X = -grad_u / vecnorm(grad_u)[:, None]

# Heat method step3: Solve Poisson Equation

In [ ]:
div_Xs = np.zeros(N)
for i1, i2, i3 in [(0, 1, 2), (1, 2, 0), (2, 0, 1)]:
    vi1, vi2, vi3 = tris[:, i1], tris[:, i2], tris[:, i3]
    e1 = verts[vi2] - verts[vi1]
    e2 = verts[vi3] - verts[vi1]
    e_oop = verts[vi3] - verts[vi2]
    cot1 = 1/np.tan(np.arccos((normalized(-e2) * normalized(-e_oop)).sum(axis = 1))) #e2_norm^T*e_oop = cos(theta)
    cot2 = 1/np.tan(np.arccos((normalized(-e1) * normalized(e_oop)).sum(axis = 1)))
    div_Xs += np.bincount(vi1.astype(int), 0.5 * (cot1*(e1*X).sum(axis = 1) + cot2 * (e2*X).sum(axis = 1)), minlength = N)

In [ ]:
VisPointsAttributes(verts, div_Xs)

# Solve Poisson equation

In [ ]:
factored_Lc = splu(Lc.tocsc())
phi = factored_Lc.solve(div_Xs)
phi -= phi.min()

In [ ]:
VisPointsAttributes(verts, phi)